In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI()

In [5]:
llm.invoke("how can langsmith help with testing?")

AIMessage(content="Langsmith can help with testing in the following ways:\n\n1. Test Automation: Langsmith can generate test scripts or test cases automatically, saving time and effort for testers. It can generate different test scenarios, inputs, and expected outputs, allowing for comprehensive test coverage.\n\n2. Test Data Generation: Langsmith can generate synthetic or realistic test data for different test scenarios. It can create a wide range of data variations, helping to uncover potential issues that might not be apparent with limited or static test data.\n\n3. Test Case Prioritization: Langsmith can analyze the code and identify high-risk areas or critical paths that require thorough testing. This helps prioritize test cases and focus on the most important functionalities or components.\n\n4. Bug Detection and Diagnosis: Langsmith can analyze code and identify potential bugs, vulnerabilities, or performance bottlenecks. It can provide insights into the root causes of issues, h

In [4]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [5]:
chain = prompt | llm

In [6]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [7]:
chain = prompt | llm | output_parser

In [8]:
llm.invoke("how can langsmith help with testing?")

AIMessage(content='Langsmith can help with testing in the following ways:\n\n1. Test Automation: Langsmith can generate test scripts automatically by analyzing the codebase. It can identify the required test cases and generate test code in various programming languages. This saves time and effort in writing test scripts manually.\n\n2. Test Coverage Analysis: Langsmith can analyze the codebase and identify areas that are not covered by existing tests. It can provide insights into the code coverage and help prioritize testing efforts.\n\n3. Test Optimization: Langsmith can analyze the existing test suite and identify redundant or duplicated test cases. It can suggest optimizations to improve the efficiency and effectiveness of the testing process.\n\n4. Test Data Generation: Langsmith can generate test data automatically based on the input parameters and expected outputs of the code. This helps in creating comprehensive test cases with different data scenarios.\n\n5. Test Case Generatio

In [9]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")

docs = loader.load()

In [10]:
from langchain_community.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [11]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()

In [12]:
documents = text_splitter.split_documents(docs)

In [13]:
vector = DocArrayInMemorySearch.from_documents(documents, embeddings)

In [14]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [15]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [17]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

LangSmith can help with testing by providing features that allow users to test changes to prompts or chains using datasets. Users can run chains over data points and visualize the outputs, allowing them to review and evaluate the results. LangSmith also provides evaluators that can be used to assess the results of test runs. Additionally, LangSmith offers annotation queues for manual review and annotation of runs, allowing users to assess subjective qualities and validate auto-evaluated runs.


In [19]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [20]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content="Skip to main content🦜️🛠️ LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.Over the past two months, we at LangChain have been building and using LangSmith with the goal of bridging this gap. This is our tactical user guide to outline effective ways to use LangSmith and maximize its benefits.On by default\u200bAt LangChain, all of us have LangSmith’s tracing running in the background by default. On the Python side, this is achieved by s

In [21]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [22]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', 'description': 'Building reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.', 'language': 'en'}),
  Document(page_content="Skip to main content🦜️🛠️ LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work n

In [23]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [25]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [26]:
tools = [retriever_tool, search]

In [29]:
from langchain.chat_models import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [30]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...
LangSmith can help with testing in several ways:

1. Test Case Generation: LangSmith can generate test cases automatically based on the specifications of the software being tested. It uses advanced techniques such as symbolic execution and constraint solving to generate test inputs that cover different execution paths and edge cases.

2. Test Oracles: LangSmith can help in defining test oracles, which are the expected outputs or behaviors of the software under test. It can analyze the specifications and requirements of the software to automatically generate assertions or checks that can be used as test oracles.

3. Test Coverage Analysis: LangSmith can analyze the code coverage of the test cases to determine which parts of the software have been exercised during testing. This helps in identifying areas of the code that have not been adequately tested and may require additional test cases.

4. Test Prioritization: LangSmith can prioritize the exec

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith can help with testing in several ways:\n\n1. Test Case Generation: LangSmith can generate test cases automatically based on the specifications of the software being tested. It uses advanced techniques such as symbolic execution and constraint solving to generate test inputs that cover different execution paths and edge cases.\n\n2. Test Oracles: LangSmith can help in defining test oracles, which are the expected outputs or behaviors of the software under test. It can analyze the specifications and requirements of the software to automatically generate assertions or checks that can be used as test oracles.\n\n3. Test Coverage Analysis: LangSmith can analyze the code coverage of the test cases to determine which parts of the software have been exercised during testing. This helps in identifying areas of the code that have not been adequately tested and may require additional test cases.\n\n4. Test Prioritization: Lan

In [31]:
agent_executor.invoke({"input": "what is the weather in SF?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'url': 'https://www.weather2travel.com/california/san-francisco/january/', 'content': 'San Francisco weather in January 2024 Expect 13°C daytime maximum temperatures  long-term weather averages for San Francisco in January before you book your next holiday to California in 2024/2025.  San Francisco January sunrise & sunset times  How sunny is it in San Francisco in January?San Francisco weather in January 2024 Expect 13°C daytime maximum temperatures in the shade with on average 6 hours of sunshine per day in San Francisco in January. Check more long-term weather averages for San Francisco in January before you book your next holiday to California in 2024/2025. 13 13°C max day temperature 6'}, {'url': 'https://www.whereandwhen.net/when/north-america/california/san-francisco-ca/january/', 'content': 'Best time to go to San Francisco? Weather in San Francisco i

{'input': 'what is the weather in SF?',
 'output': 'The weather in San Francisco is currently not available. However, the average temperature in January is around 13°C (55°F) during the day. It is recommended to check a reliable weather website or app for the most up-to-date information.'}

In [32]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})



> Entering new AgentExecutor chain...
LangSmith can help test your LLM applications in the following ways:

1. Personal Statement Review: LangSmith can provide feedback and suggestions on your personal statement to ensure it effectively communicates your motivations, experiences, and goals. This includes checking for clarity, coherence, and overall impact.

2. Resume/CV Review: LangSmith can review your resume or CV to ensure it is well-structured, highlights your relevant experiences and achievements, and effectively presents your qualifications.

3. Recommendation Letter Guidance: LangSmith can provide guidance on selecting appropriate recommenders and offer suggestions on what information to provide them to ensure strong and compelling recommendation letters.

4. Application Strategy: LangSmith can help you develop a comprehensive application strategy, including selecting the right law schools, understanding their specific requirements, and organizing your application materials.



{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'output': 'LangSmith can help test your LLM applications in the following ways:\n\n1. Personal Statement Review: LangSmith can provide feedback and suggestions on your personal statement to ensure it effectively communicates your motivations, experiences, and goals. This includes checking for clarity, coherence, and overall impact.\n\n2. Resume/CV Review: LangSmith can review your resume or CV to ensure it is well-structured, highlights your relevant experiences and achievements, and effectively presents your qualifications.\n\n3. Recommendation Letter Guidance: LangSmith can provide guidance on selecting appropriate recommenders and offer suggestions on what information to provide them to ensure strong and compelling recommendation letters.\n\n4. Application Strategy: LangSmith can help you develop a comprehensive application strategy, includi